In [1]:
import tensorflow as tf 
import numpy as np

# 数据输入：tf.data

In [46]:
# 加载数据集：只用训练集即可
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

In [47]:
# 维度拓展，从数组转为图像尺寸
train_image = tf.expand_dims(train_image, -1)
test_image = tf.expand_dims(test_image, -1)
train_image.shape

TensorShape([60000, 28, 28, 1])

In [48]:
# 转换数据类型：
train_image = tf.cast( train_image/255, tf.float32 )  # 归一化后，转为float32
train_label = tf.cast( train_label, tf.int32 )

test_image = tf.cast( test_image/255, tf.float32 )
test_label = tf.cast( test_label, tf.int32 )

In [49]:
# tf.data进行数据集输入：
train_dataset = tf.data.Dataset.from_tensor_slices( (train_image,train_label) )
test_dataset = tf.data.Dataset.from_tensor_slices( (test_image,test_label) )
train_dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int32)>

In [50]:
# 数据集乱序、分批次：
train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.shuffle(10000).batch(64)

# 网络搭建：tf.keras.xxx

In [51]:
model = tf.keras.Sequential( [
    tf.keras.layers.Conv2D( 16, [3,3], activation = 'relu', input_shape = (None, None, 1) ),  # input_shape这样写即任何输入尺寸都行！
    tf.keras.layers.Conv2D( 32, [3,3], activation = 'relu' ),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)   #  其实可以不用softmax激活到[0,1]概率值的，哪个分数最高就是哪个！ 
] )

In [52]:
# 自定义优化器对象：
optimizer = tf.keras.optimizers.Adam( lr = 0.001 )

In [53]:
# 自定义损失函数对象：
loss_func = tf.keras.losses.SparseCategoricalCrossentropy( from_logits = True )  # 前面Dense没有用softmax激活，这样要告诉一下！

In [54]:
# 定义指标计算“对象”：目标是求每个batch的！！ —— 故：加到train_step里。—— 公用对象，一直在变！！！★★★
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_acc')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_acc')

# 自定义训练函数、指标计算函数

In [55]:
# 没用：
# 定义损失函数：
# 输入：模型、训练图像、真实标签；功能：计算网络对该图像的预测值（预测标签）；返回：预测值与真实值间的损失
# def loss(model, images, labels_real):
#     labels_predict = model(images)  # 网络的预测
#     return loss_func(labels_real, labels_predict)  # 返回二者的交叉熵损失

In [56]:
# 每个batch的训练函数：
def train_step(model, images, labels_real):
    
    with tf.GradientTape() as t:  # 定义t要跟踪哪些函数的梯度变化！
        labels_predict = model(images)                      # 当前model对图像的预测
        loss_step = loss_func(labels_real, labels_predict)  # 直接用
        
    # 梯度优化：
    grads = t.gradient( loss_step, model.trainable_variables )  # 求目标函数关于“各个”可训练参数的梯度值！
    # 用上一步得到“各个”可训练参数梯度值，修改“各个”可训练参数，即也就修改了model
    optimizer.apply_gradients( zip(grads, model.trainable_variables) )  
    
    # 每个batch指标计算：64张训练图片的评价loss和acc —— 公用可调用对象，一直在变！★★★
    train_loss( loss_step )  # 每个batch中所有图的误差的均值
    train_accuracy( labels_real, labels_predict )

In [63]:
# 一般每次epoch测试一次就行了，不用每个batch那么频繁！
def test_step(model, images, labels_real):
    labels_predict = model(images)
    loss_step = loss_func(labels_real, labels_predict)
    
    # 每个batch指标计算：64张测试图片的评价loss和acc —— 公用可调用对象，一直在变！★★★
    test_loss( loss_step )
    test_accuracy( labels_real, labels_predict )

In [66]:
# 总体训练的执行函数：注函数！
def train():
    
    # 每个epoch大循环：
    for epoch in range(10):
        # 每个batch小循环：
        for (batch, (images, labels_real)) in enumerate(train_dataset):  # 每次拿一个单位（batch）出来，带上编号
            train_step( model, images, labels_real )  # 每个batch的训练！
            # 在for循环内，也就是每个batch都在打印！
            print( 'Epoch{}_batch{}：train_loss：{}，train_acc：{}'.format(epoch+1, 
                                                                           batch, 
                                                                           train_loss.result().numpy(), 
                                                                           train_accuracy.result().numpy()) )
            
        # 一直在累积：内部for循环每完毕一次，就是一个epoch完成，打印
        print('Epoch{}：train_loss：{}，train_acc：{}'.format(epoch+1,
                                                              train_loss.result().numpy(), 
                                                              train_accuracy.result().numpy()) )
        
        # 每个epoch测试一下当前模型“预测”能力：
        for (batch, (images, labels_real)) in enumerate(test_dataset):
            test_step( model, images, labels_real )
        print('Epoch{}：test_loss：{}，test_acc：{}'.format(epoch+1,
                                                            test_loss.result().numpy(),
                                                            test_accuracy.result().numpy()) )
        
        # 每个epoch所有训练集完毕，需要把对象重置：
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [67]:
train()

Epoch1_batch0：train_loss：2.1725924015045166，train_acc：0.19694368541240692
Epoch1_batch1：train_loss：2.1721155643463135，train_acc：0.1969662457704544
Epoch1_batch2：train_loss：2.171412706375122，train_acc：0.19727273285388947
Epoch1_batch3：train_loss：2.1705548763275146，train_acc：0.19774682819843292
Epoch1_batch4：train_loss：2.1698434352874756，train_acc：0.1977098435163498
Epoch1_batch5：train_loss：2.1692054271698，train_acc：0.19801034033298492
Epoch1_batch6：train_loss：2.1683592796325684，train_acc：0.19858871400356293
Epoch1_batch7：train_loss：2.167645215988159，train_acc：0.19899553060531616
Epoch1_batch8：train_loss：2.1665189266204834，train_acc：0.19934386014938354
Epoch1_batch9：train_loss：2.165571928024292，train_acc：0.19968971610069275
Epoch1_batch10：train_loss：2.1647560596466064，train_acc：0.19975706934928894
Epoch1_batch11：train_loss：2.163816452026367，train_acc：0.1999339759349823
Epoch1_batch12：train_loss：2.1627390384674072，train_acc：0.2003837674856186
Epoch1_batch13：train_loss：2.161750316619873，tr

KeyboardInterrupt: 

# 总结

说明的所有自定义函数，其功能完成已经和tf.keras封装好的高级操作一样了！
- tf.keras会用3大模块：tf.keras.layers、model.compile、model.fit
- 自定义用到的可调用对象：optimizer、loss_func；train_loss、train_accuracy；test_func、test_accuracy
- 自定义用到的函数：train_step、test_step、train

注意
- 在使用自定义搭建时，用实例化函数创建的都是“**可调用对象**”！！例如：optimizer、loss_func；train_loss、train_accuracy；test_func、test_accuracy都是！—— 不论何时使用，它们都有“**自动记录**”功能！
- 用tf.keras高阶模块搭建时，dataset数据需要.repeat()；当是自定义循环时，不需要用.repeat()